## Reto 2: Convertir tablas en MySQL a `DataFrames` de `pandas`

### 1. Objetivos:
    - Solicitar todos los datos de las tablas que están almacenadas en nuestra base de datos, convertirlos a `DataFrames` y guardarlos.
    
---
    
### 2. Desarrollo:

#### a) Tablas a `DataFrames`

Ya que tenemos nuestra conexión funcionando adecuadamente, vamos a utilizarla para realizar consultas a las base de datos y construir una base de datos local. Tu Reto consiste en los siguientes pasos:

1. Vuelve a establecer la conexión a la base de datos
2. Usando el comando `SELECT * FROM nombre_de_tabla`, realiza consultas a cada una de las 5 tablas que existen en la base de datos.
3. Crea un `DataFrame` por cada tabla que obtuviste. Para asignarle los nombres de las columnas correctamente, revisa el archivo [Readme.md](../../Datasets/MovieLens/Readme.md) donde está contenida toda esa información.
4. Asegúrate de que el índice sea adecuado en cada `DataFrame`. En los casos en los que haya datos redundantes, convierte una de las columnas en índice.
5. Si lo deseas ordena las columnas de la manera en la que mejor te parezca.
6. Guarda tus `DataFrames` en formato .csv para utilizarlos en los siguientes Retos.

> **NOTA IMPORTANTE**: La tabla movies es un poco complicada porque contiene muchos signos distintos. Tanto en la columna de nombre de película como la de género, encontramos signos como `,`, `:`, `.`, `|`. Esto hace un poco complicado el almacenamiento y lectura de este archivo. Si elijes guardar este archivo como un .csv separado por comas (`,`), a la hora de leerlo de regreso, `pandas` puede confundirse y pensar que el título de una película que contiene comas constituye dos columnas. Por esta razón, te recomiendo que la tabla `movies` la guardes agregando un separador poco convencional como `sep='$'`. De esta manera será muchísimo más fácil leer tu archivo de regreso usando ese separador.

In [1]:
!pip install mysql-connector-python

     |████████████████████████████████| 18.1MB 253kB/s 


In [3]:
import mysql.connector

cnx = mysql.connector.connect(
    host = "ec2-34-222-38-19.us-west-2.compute.amazonaws.com",
    port = 3306,
    user = "data",
    password = "passwd",
    database='movielens'
)

cursor = cnx.cursor()

# Passwd = khQ9--mp6LWq97iA

In [8]:
cursor.execute("SHOW TABLES")
cursor.fetchall()

[('age_ranges',), ('movies',), ('occupations',), ('ratings',), ('users',)]

In [24]:
def get_table_as_df(cursor, table, columns, index_col=None):
    #df =  pd.read_sql('SELECT * FROM <table>', cnx)
    import pandas as pd
    try:
      cursor.execute(f"SELECT * FROM {table}")
      rs = cursor.fetchall()
    except mysql.connector.Error as err:
      print("Something went wrong: {}".format(err))
      return
    df = pd.DataFrame(rs, columns=columns)
    return df.set_index(index_col, drop=True) if index_col else df

In [25]:
ages = get_table_as_df(cursor, "age_ranges", ["age_id", "age_range"], "age_id")
movies = get_table_as_df(cursor, "movies", ["movie_id", "title", "genres"], "movie_id")
occup = get_table_as_df(cursor, "occupations", ["occupation_id", "description"], "occupation_id")
ratings = get_table_as_df(cursor, "ratings", ["user_id", "movie_id", "rating", "timestamp"])
users = get_table_as_df(cursor, "users", ['user_id', 'gender', 'age', 'occupation', 'cp'], "user_id")

In [29]:
cursor.close()

True

In [ ]:
ages.to_csv("ages.csv", sep="$", encoding="utf-8")
movies.to_csv("movies.csv", sep="$", encoding="utf-8")
occup.to_csv("occupation.csv", sep="$", encoding="utf-8")
ratings.to_csv("occupation.csv", sep="$", encoding="utf-8", index=False)
users.to_csv("occupation.csv", sep="$", encoding="utf-8")

Compara con tus compañeros y revisa con la experta para que todos estén seguros de que tienen sus `DataFrames` estructurados de la manera correcta y que sus archivos .csv fueron creados exitosamente. Vamos a utilizar estos archivos en los Retos siguientes, así que es muy importante que tus datos estén estructurados adecuadamente.